# Voice Synthesize with WaveGlow Model in Coogle CoLAB

by Hyungon Ryu | Sr. Solution Architect


I'll demonstrate real-time inferencing WaveGlow

## allocate GPU

check GPU is available in your notebook


 in **[EDIT]** Menu> Select **[Notebook Settings]** > check ** [ GPU ]**

In [1]:
!nvidia-smi

Thu Nov  8 13:32:00 2018       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 396.44                 Driver Version: 396.44                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P8    30W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

## Devops

for WaveGlow, we need **pytorch 0.4.0** with CUDA 9.0
**(TODO : I'll update pytorch 0.4.1+ in CUDA 10)**


### install  pytorch 0.4.0

for **Tacotron2** and **WaveGlow** dependency, we need **librosa and other libraries 

In [0]:
!pip install -q  matplotlib==2.1.0 

In [3]:
!pip install numpy inflect librosa scipy tensorboardX Unidecode pillow

    100% |████████████████████████████████| 61kB 5.2MB/s 
    100% |████████████████████████████████| 1.6MB 16.8MB/s 
    100% |████████████████████████████████| 71kB 22.8MB/s 
    100% |████████████████████████████████| 235kB 29.0MB/s 
    100% |████████████████████████████████| 327kB 23.5MB/s 
    100% |████████████████████████████████| 3.2MB 11.0MB/s 
    100% |████████████████████████████████| 16.1MB 2.6MB/s 
  Running setup.py bdist_wheel for librosa ... - \ done
  Stored in directory: /root/.cache/pip/wheels/18/b8/10/f0f8f6ac60668a5cd75596cf14c25bb6b3ea1ecd815f058b7e
  Running setup.py bdist_wheel for audioread ... - done
  Stored in directory: /root/.cache/pip/wheels/53/02/90/7b5c4081b7470c550ab605f600bad237dde12a6b8999b11f50
  Running setup.py bdist_wheel for resampy ... - \ done
  Stored in directory: /root/.cache/pip/wheels/ff/4f/ed/2e6c676c23efe5394bb40ade50662e90eb46e29b48324c5f9b
Successfully built librosa audioread resampy


## clone WaveGlow Model
I'm using my repository. I strongly recommend to use NVIDIA's official git repository 


In [4]:
#!git clone https://github.com/NVIDIA/waveglow.git
!git clone https://github.com/yhgon/waveglow.git

Cloning into 'waveglow'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 26 (delta 9), reused 10 (delta 2), pack-reused 0
Unpacking objects: 100% (26/26), done.


In [6]:
!ls -alh waveglow

total 3.2M
drwxr-xr-x 5 root root 4.0K Nov  8 12:10 .
drwxr-xr-x 1 root root 4.0K Nov  8 12:08 ..
-rwxr-xr-x 1 root root  881 Nov  8 10:52 config.json
-rw-r--r-- 1 root root 7.3K Nov  8 10:52 distributed.py
drwxr-xr-x 9 root root 4.0K Nov  8 12:01 .git
-rw-r--r-- 1 root root   84 Nov  8 10:52 .gitmodules
-rw-r--r-- 1 root root 5.9K Nov  8 10:52 glow_old.py
-rw-r--r-- 1 root root  13K Nov  8 10:52 glow.py
-rw-r--r-- 1 root root 3.5K Nov  8 10:52 inference.py
-rw-r--r-- 1 root root 399K Nov  8 12:09 LJ001-0015.wav_synthesis.wav
-rw-r--r-- 1 root root 210K Nov  8 12:09 LJ001-0051.wav_synthesis.wav
-rw-r--r-- 1 root root 198K Nov  8 12:09 LJ001-0063.wav_synthesis.wav
-rw-r--r-- 1 root root 281K Nov  8 12:10 LJ001-0072.wav_synthesis.wav
-rw-r--r-- 1 root root 191K Nov  8 12:10 LJ001-0079.wav_synthesis.wav
-rw-r--r-- 1 root root 186K Nov  8 12:10 LJ001-0094.wav_synthesis.wav
-rw-r--r-- 1 root root 417K Nov  8 12:10 LJ001-0096.wav_synthesis.wav
-rw-r--r-- 1 root root 223K Nov  8 12:10 LJ001-0

## install tacotron2

to match audio file preprocessing, we need to  use tacotron2.
you could update tacotron2 repository in submodule

In [5]:
%%bash
pwd
cd waveglow
git submodule init
git submodule update
ls -lah
ls -alh tacotron2

/content
Submodule path 'tacotron2': checked out 'fc0cf6a89a47166350b65daa1beaa06979e4cddf'
total 1.3M
drwxr-xr-x 4 root root 4.0K Nov  8 13:33 .
drwxr-xr-x 1 root root 4.0K Nov  8 13:33 ..
-rwxr-xr-x 1 root root  881 Nov  8 13:33 config.json
-rw-r--r-- 1 root root 7.3K Nov  8 13:33 distributed.py
drwxr-xr-x 9 root root 4.0K Nov  8 13:33 .git
-rw-r--r-- 1 root root   84 Nov  8 13:33 .gitmodules
-rw-r--r-- 1 root root 5.9K Nov  8 13:33 glow_old.py
-rw-r--r-- 1 root root  13K Nov  8 13:33 glow.py
-rw-r--r-- 1 root root 3.5K Nov  8 13:33 inference.py
-rw-r--r-- 1 root root 5.7K Nov  8 13:33 mel2samp.py
-rwxr-xr-x 1 root root 2.7K Nov  8 13:33 README.md
drwxr-xr-x 4 root root 4.0K Nov  8 13:33 tacotron2
-rw-r--r-- 1 root root 7.3K Nov  8 13:33 train.py
-rw-r--r-- 1 root root 860K Nov  8 13:33 TTS_inference_WaveGlow_in_COLAB.ipynb
-rwxr-xr-x 1 root root 382K Nov  8 13:33 waveglow_logo.png
total 1.2M
drwxr-xr-x 4 root root 4.0K Nov  8 13:33 .
drwxr-xr-x 4 root root 4.0K Nov  8 13:33 ..
-rw-r

Submodule 'tacotron2' (http://github.com/NVIDIA/tacotron2) registered for path 'tacotron2'
Cloning into '/content/waveglow/tacotron2'...


In [0]:
!pwd

## Download Weight 

there are two options.  I strongly recommend to copy to google drive to reuse. 

 - colab drive
 
  - step1. copy to google drive
  - step2. mount Google drive
  - step3. copy to local disk in COLAB

- direct download
 
 -step1. download file to COLAB disk
 

 

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


I copy the weight files and mel files in COLAB_data folder.
change right folder 

In [7]:
%%time
!ls -alh   "drive/My Drive/COLAB_data"

total 2.0G
dr-x------ 2 root root 4.0K Jul 28  2016 CelebA
dr-x------ 2 root root 4.0K Nov 11  2017 celeba-hq-deltas
-r-------- 2 root root 1.5M Nov  8 00:41 mel_spectrograms.zip
drwx------ 2 root root 4.0K Oct  4 10:39 segan
-r-------- 2 root root 2.0G Nov  8 03:51 waveglow_old.pt
CPU times: user 48.8 ms, sys: 5.55 ms, total: 54.3 ms
Wall time: 3.99 s


It takes 32 sec to copy 2GB file from google drive to COLAB  

In [8]:
%%time
!cp -rf "drive/My Drive/COLAB_data/waveglow_old.pt" .

CPU times: user 165 ms, sys: 27.2 ms, total: 192 ms
Wall time: 31 s


##  Mel Spectrum


copy Mel file from google Drive
(TODO ) add live Mel Spectrum Generation from text with Tacotron2


In [11]:
%%time
!cp -rf "drive/My Drive/COLAB_data/mel_spectrograms.zip" .

CPU times: user 27.7 ms, sys: 8.21 ms, total: 35.9 ms
Wall time: 2.22 s


### decompress Mel files

In [0]:
!unzip mel_spectrograms.zip

I found file name have some issue because original file is compressed in MACOS.
so eliminate the folder  and decompress agin excluding  MACOS related files  

https://github.com/NVIDIA/waveglow/issues/3

In [0]:
!rm -rf content/mel_spectrogram/.DS_Store

if you use your own repository and modified code,  update  

In [0]:
%%bash
cd waveglow
git pull

# Inferencing

it takes few seconds  to load weight files(2GB) and synthesize voice.

In [14]:
%%bash
pwd
cd waveglow

python inference.py  -f <(ls /content/mel_spectrograms/*.pt)  -w /content/waveglow_old.pt -o . -s 0.6


/content
./LJ001-0015.wav_synthesis.wav
./LJ001-0051.wav_synthesis.wav
./LJ001-0063.wav_synthesis.wav
./LJ001-0072.wav_synthesis.wav
./LJ001-0079.wav_synthesis.wav
./LJ001-0094.wav_synthesis.wav
./LJ001-0096.wav_synthesis.wav
./LJ001-0102.wav_synthesis.wav
./LJ001-0153.wav_synthesis.wav
./LJ001-0173.wav_synthesis.wav


## listen voice 
You could  compare original LJSpeech dataset

In [15]:
import librosa.display
import IPython
from IPython.display import Audio
IPython.display.display(Audio('/content/waveglow/LJ001-0015.wav_synthesis.wav', rate=22050))
IPython.display.display(Audio('/content/waveglow/LJ001-0051.wav_synthesis.wav', rate=22050))

## 

### (TODO) real-time inferencing with Tacotron Weight + WaveGlow model.
 - Step1 DevOps
 
 - Step2. clone Model and download weight files
 
 - Step3 load Model param
 
  -Tacotron2 weight
  - WaveGlow weight

- Step4. input text 

- Step5. Generate Mel 
- Step6. Generate Wav
- Step7. display wavefile
